<a href="https://colab.research.google.com/github/DanielAndresSmg/Home-Credit-Default-Risk/blob/main/Chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# installments_payments (Chapter 2)

In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import gc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
d1 = pd.read_csv('/content/drive/MyDrive/Dataset/Project/datatrain.csv')
d5 = pd.read_csv('/content/drive/MyDrive/Dataset/Project/installments_payments.csv')
d7 = pd.read_csv('/content/drive/MyDrive/Dataset/Project/POS_CASH_balance.csv')
d8 = pd.read_csv('/content/drive/MyDrive/Dataset/Project/previous_application.csv')

In [4]:
d5['DAYS_DIFF'] = d5['DAYS_INSTALMENT'] - d5['DAYS_ENTRY_PAYMENT']
d5['AMT_DIFF'] = d5['AMT_INSTALMENT'] - d5['AMT_PAYMENT']
d5.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DAYS_DIFF,AMT_DIFF
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,7.0,0.000
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,0.0,0.000
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,0.0,0.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,8.0,0.000
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,-17.0,4.455


In [5]:
install_stats_by_prev = d5[['SK_ID_PREV', 'SK_ID_CURR']]

In [6]:
install_NUM_INSTALMENT_VERSION_count = d5.groupby('SK_ID_PREV', as_index=False)['NUM_INSTALMENT_VERSION'].count().rename(columns = {'NUM_INSTALMENT_VERSION': 'install_NUM_INSTALMENT_VERSION_count'})
install_NUM_INSTALMENT_VERSION_max = d5.groupby('SK_ID_PREV', as_index=False)['NUM_INSTALMENT_VERSION'].max().rename(columns = {'NUM_INSTALMENT_VERSION': 'install_NUM_INSTALMENT_VERSION_max'})
install_stats_by_prev = install_stats_by_prev.merge(install_NUM_INSTALMENT_VERSION_count, on = 'SK_ID_PREV', how = 'left')
install_stats_by_prev = install_stats_by_prev.merge(install_NUM_INSTALMENT_VERSION_max, on = 'SK_ID_PREV', how = 'left')

In [7]:
gc.enable()
del install_NUM_INSTALMENT_VERSION_count, install_NUM_INSTALMENT_VERSION_max
gc.collect()

0

In [8]:
# Function calculate mean colomn
def calculate_mean(d,df, col, new_col):
    # group by SK_ID_PREV and calculate mean of the selected column
    temp = d.groupby('SK_ID_PREV', as_index=False)[col].mean().rename(columns = {col: new_col})
    # merge the result with the original data frame
    df = df.merge(temp, on = 'SK_ID_PREV', how = 'left')
    # delete the temporary variable
    del temp
    # run garbage collector
    gc.collect()
    # return the merged data frame
    return df

In [9]:
install_stats_by_prev = calculate_mean(d5,install_stats_by_prev, 'DAYS_INSTALMENT', 'install_DAYS_INSTALMENT_mean')
install_stats_by_prev = calculate_mean(d5,install_stats_by_prev, 'DAYS_ENTRY_PAYMENT', 'install_DAYS_ENTRY_PAYMENT_mean')
install_stats_by_prev = calculate_mean(d5,install_stats_by_prev, 'AMT_INSTALMENT', 'install_AMT_INSTALMENT_mean')
install_stats_by_prev = calculate_mean(d5,install_stats_by_prev, 'AMT_PAYMENT', 'install_AMT_PAYMENT_mean')
install_stats_by_prev = calculate_mean(d5,install_stats_by_prev, 'DAYS_DIFF', 'install_DAYS_DIFF_mean')

In [10]:
install_DAYS_DIFF_max = d5.groupby('SK_ID_PREV', as_index=False)['DAYS_DIFF'].max().rename(columns = {'DAYS_DIFF': 'install_DAYS_DIFF_max'})
install_DAYS_DIFF_min = d5.groupby('SK_ID_PREV', as_index=False)['DAYS_DIFF'].min().rename(columns = {'DAYS_DIFF': 'install_DAYS_DIFF_min'})
install_stats_by_prev = install_stats_by_prev.merge(install_DAYS_DIFF_max, on = 'SK_ID_PREV', how = 'left')
install_stats_by_prev = install_stats_by_prev.merge(install_DAYS_DIFF_min, on = 'SK_ID_PREV', how = 'left')
gc.enable()
del install_DAYS_DIFF_max, install_DAYS_DIFF_min
gc.collect()

0

In [11]:
# capture the mean, max, and min for AMT_DIFF
install_AMT_DIFF_mean = d5.groupby('SK_ID_PREV', as_index=False)['AMT_DIFF'].mean().rename(columns = {'AMT_DIFF': 'install_AMT_DIFF_mean'})
install_AMT_DIFF_max = d5.groupby('SK_ID_PREV', as_index=False)['AMT_DIFF'].max().rename(columns = {'AMT_DIFF': 'install_AMT_DIFF_max'})
install_AMT_DIFF_min = d5.groupby('SK_ID_PREV', as_index=False)['AMT_DIFF'].min().rename(columns = {'AMT_DIFF': 'install_AMT_DIFF_min'})
install_stats_by_prev = install_stats_by_prev.merge(install_AMT_DIFF_mean, on = 'SK_ID_PREV', how = 'left')
install_stats_by_prev = install_stats_by_prev.merge(install_AMT_DIFF_max, on = 'SK_ID_PREV', how = 'left')
install_stats_by_prev = install_stats_by_prev.merge(install_AMT_DIFF_min, on = 'SK_ID_PREV', how = 'left')
gc.enable()
del install_AMT_DIFF_mean, install_AMT_DIFF_max, install_AMT_DIFF_min
gc.collect()

0

In [12]:
install_stats_by_prev.head()

,SK_ID_PREV,SK_ID_CURR,install_NUM_INSTALMENT_VERSION_count,install_NUM_INSTALMENT_VERSION_max,install_DAYS_INSTALMENT_mean,install_DAYS_ENTRY_PAYMENT_mean,install_AMT_INSTALMENT_mean,install_AMT_PAYMENT_mean,install_DAYS_DIFF_mean,install_DAYS_DIFF_max,install_DAYS_DIFF_min,install_AMT_DIFF_mean,install_AMT_DIFF_max,install_AMT_DIFF_min
0,1054186,161674,12,1.0,-1165.000000,-1191.333333,6947.265000,6947.265000,26.333333,65.0,5.0,0.00000,0.000,0.0
1,1330831,151639,103,0.0,-1544.660194,-1547.048544,2358.188301,2325.421311,2.388350,29.0,-5.0,32.76699,2250.000,0.0
2,2085231,193053,3,3.0,-40.333333,-35.000000,11483.070000,11483.070000,-5.333333,7.0,-23.0,0.00000,0.000,0.0
3,2452527,199697,6,1.0,-2403.000000,-2411.000000,24332.977500,24332.977500,8.000000,11.0,5.0,0.00000,0.000,0.0
4,2714724,167756,6,2.0,-1368.000000,-1374.333333,3419.415000,2697.735000,6.333333,38.0,-17.0,721.68000,2160.585,0.0


In [13]:
install_NUM_INSTALMENT_VERSION_max_max = install_stats_by_prev.groupby('SK_ID_CURR', as_index=False)['install_NUM_INSTALMENT_VERSION_max'].max().rename(columns = {'install_NUM_INSTALMENT_VERSION_max': 'install_NUM_INSTALMENT_VERSION_max_max'})
d1 = d1.merge(install_NUM_INSTALMENT_VERSION_max_max, on = 'SK_ID_CURR', how = 'left')
gc.enable()
del install_NUM_INSTALMENT_VERSION_max_max
gc.collect()

0

In [14]:
# Define a function that takes a column name and returns the mean of that column grouped by SK_ID_CURR
def install_mean(column):
  return install_stats_by_prev.groupby('SK_ID_CURR', as_index=False)[column].mean().rename(columns = {column: column + '_mean'})

# Define a list of columns to apply the function to
var_list = ['install_NUM_INSTALMENT_VERSION_count',
            'install_NUM_INSTALMENT_VERSION_max',
            'install_DAYS_INSTALMENT_mean',
            'install_DAYS_ENTRY_PAYMENT_mean',
            'install_AMT_INSTALMENT_mean',
            'install_AMT_PAYMENT_mean',
            'install_DAYS_DIFF_mean',
            'install_DAYS_DIFF_max',
            'install_DAYS_DIFF_min',
            'install_AMT_DIFF_mean',
            'install_AMT_DIFF_max',
            'install_AMT_DIFF_min']

for column in var_list:
  d1 = d1.merge(install_mean(column), on = 'SK_ID_CURR', how = 'left')


In [15]:
print('dataset data shape: ', d1.shape)
d1.head()

dataset data shape:  (307507, 193)


,Unnamed: 0,SK_ID_CURR,DAYS_EMPLOYED,NAME_CONTRACT_TYP,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,install_DAYS_INSTALMENT_mean_mean,install_DAYS_ENTRY_PAYMENT_mean_mean,install_AMT_INSTALMENT_mean_mean,install_AMT_PAYMENT_mean_mean,install_DAYS_DIFF_mean_mean,install_DAYS_DIFF_max_mean,install_DAYS_DIFF_min_mean,install_AMT_DIFF_mean_mean,install_AMT_DIFF_max_mean,install_AMT_DIFF_min_mean
0,0,100002,-637,NaN,1,0,1,0,202500.0,406597.5,...,-295.000000,-315.421053,11559.247105,11559.247105,20.421053,31.000000,12.000000,0.000000,0.000000,0.0
1,1,100003,-1188,NaN,0,0,0,0,270000.0,1293502.5,...,-1378.160000,-1385.320000,64754.586000,64754.586000,7.160000,12.320000,3.480000,0.000000,0.000000,0.0
2,2,100004,-225,NaN,1,1,1,0,67500.0,135000.0,...,-754.000000,-761.666667,7096.155000,7096.155000,7.666667,11.000000,3.000000,0.000000,0.000000,0.0
3,3,100006,-3039,NaN,0,0,1,0,135000.0,312682.5,...,-252.250000,-271.625000,62947.088437,62947.088437,19.375000,33.625000,6.125000,0.000000,0.000000,0.0
4,4,100007,-3038,NaN,1,0,1,0,121500.0,513000.0,...,-1028.606061,-1032.242424,12666.444545,12214.060227,3.636364,18.212121,-3.181818,452.384318,5225.751136,0.0


# POS_CASH_balance

In [16]:
cash_stats_by_prev = d7[['SK_ID_PREV', 'SK_ID_CURR']]

In [17]:
cash_MONTHS_BALANCE_count = d7.groupby('SK_ID_PREV', as_index=False)['MONTHS_BALANCE'].count().rename(columns = {'MONTHS_BALANCE': 'cash_MONTHS_BALANCE_count'})
cash_MONTHS_BALANCE_mean = d7.groupby('SK_ID_PREV', as_index=False)['MONTHS_BALANCE'].mean().rename(columns = {'MONTHS_BALANCE': 'cash_MONTHS_BALANCE_mean'})
cash_stats_by_prev = cash_stats_by_prev.merge(cash_MONTHS_BALANCE_count, on = 'SK_ID_PREV', how = 'left')
cash_stats_by_prev = cash_stats_by_prev.merge(cash_MONTHS_BALANCE_mean, on = 'SK_ID_PREV', how = 'left')
gc.enable()
del cash_MONTHS_BALANCE_count, cash_MONTHS_BALANCE_mean
gc.collect()

0

In [18]:
cash_stats_by_prev = calculate_mean(d7,cash_stats_by_prev, 'CNT_INSTALMENT', 'cash_CNT_INSTALMENT_mean')
cash_stats_by_prev = calculate_mean(d7,cash_stats_by_prev, 'CNT_INSTALMENT_FUTURE', 'cash_CNT_INSTALMENT_FUTURE_mean')
cash_stats_by_prev = calculate_mean(d7,cash_stats_by_prev, 'SK_DPD', 'cash_SK_DPD_mean')
cash_stats_by_prev = calculate_mean(d7,cash_stats_by_prev, 'SK_DPD_DEF', 'cash_SK_DPD_DEF_mean')

In [19]:
# now let's deal with our one categorical variable, NAME_CONTRACT_STATUS, in this cash file
cash_cats = pd.get_dummies(d7.select_dtypes('object'))
cash_cats['SK_ID_PREV'] = d7['SK_ID_PREV']
cash_cats.head()

,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA,SK_ID_PREV
0,1,0,0,0,0,0,0,0,0,1803195
1,1,0,0,0,0,0,0,0,0,1715348
2,1,0,0,0,0,0,0,0,0,1784872
3,1,0,0,0,0,0,0,0,0,1903291
4,1,0,0,0,0,0,0,0,0,2341044


In [20]:
cash_cats_grouped = cash_cats.groupby('SK_ID_PREV').agg('sum')
cash_cats_grouped.head()

,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
SK_ID_PREV,,,,,,,,,
1000001,2,0,0,0,1,0,0,0,0
1000002,4,0,0,0,1,0,0,0,0
1000003,4,0,0,0,0,0,0,0,0
1000004,7,0,0,0,1,0,0,0,0
1000005,10,0,0,0,1,0,0,0,0


In [21]:
cash_stats_by_prev = cash_stats_by_prev.merge(cash_cats_grouped, on = 'SK_ID_PREV', how = 'left')
gc.enable()
del cash_cats_grouped, cash_cats
gc.collect()

0

In [22]:
print('dataset data shape: ', d1.shape)
d1.head()

dataset data shape:  (307507, 193)


,Unnamed: 0,SK_ID_CURR,DAYS_EMPLOYED,NAME_CONTRACT_TYP,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,install_DAYS_INSTALMENT_mean_mean,install_DAYS_ENTRY_PAYMENT_mean_mean,install_AMT_INSTALMENT_mean_mean,install_AMT_PAYMENT_mean_mean,install_DAYS_DIFF_mean_mean,install_DAYS_DIFF_max_mean,install_DAYS_DIFF_min_mean,install_AMT_DIFF_mean_mean,install_AMT_DIFF_max_mean,install_AMT_DIFF_min_mean
0,0,100002,-637,NaN,1,0,1,0,202500.0,406597.5,...,-295.000000,-315.421053,11559.247105,11559.247105,20.421053,31.000000,12.000000,0.000000,0.000000,0.0
1,1,100003,-1188,NaN,0,0,0,0,270000.0,1293502.5,...,-1378.160000,-1385.320000,64754.586000,64754.586000,7.160000,12.320000,3.480000,0.000000,0.000000,0.0
2,2,100004,-225,NaN,1,1,1,0,67500.0,135000.0,...,-754.000000,-761.666667,7096.155000,7096.155000,7.666667,11.000000,3.000000,0.000000,0.000000,0.0
3,3,100006,-3039,NaN,0,0,1,0,135000.0,312682.5,...,-252.250000,-271.625000,62947.088437,62947.088437,19.375000,33.625000,6.125000,0.000000,0.000000,0.0
4,4,100007,-3038,NaN,1,0,1,0,121500.0,513000.0,...,-1028.606061,-1032.242424,12666.444545,12214.060227,3.636364,18.212121,-3.181818,452.384318,5225.751136,0.0


In [23]:
# Define a function that takes a column name and returns the mean of that column grouped by SK_ID_CURR
def cash_mean(column):
  return cash_stats_by_prev.groupby('SK_ID_CURR', as_index=False)[column].mean().rename(columns = {column: column + '_mean'})

# Define a list of columns to apply the function to
var_list = ['cash_MONTHS_BALANCE_count',
            'cash_MONTHS_BALANCE_mean',
            'cash_CNT_INSTALMENT_mean',
            'cash_CNT_INSTALMENT_FUTURE_mean',
            'cash_SK_DPD_mean',
            'cash_SK_DPD_DEF_mean',
            'NAME_CONTRACT_STATUS_Active',
            'NAME_CONTRACT_STATUS_Amortized debt',
            'NAME_CONTRACT_STATUS_Approved',
            'NAME_CONTRACT_STATUS_Canceled',
            'NAME_CONTRACT_STATUS_Completed',
            'NAME_CONTRACT_STATUS_Demand',
            'NAME_CONTRACT_STATUS_Returned to the store',
            'NAME_CONTRACT_STATUS_Signed',
            'NAME_CONTRACT_STATUS_XNA']

for column in var_list:
  d1 = d1.merge(cash_mean(column), on = 'SK_ID_CURR', how = 'left')
# let's free up some memory by deleting some of the dataframes we are done with

In [ ]:
print('dataset data shape: ', d1.shape)
d1.drop(d1.columns[[0]], axis = 1, inplace = True)
d1.head()

In [ ]:
# let's free up some memory by deleting some of the dataframes we are done with
gc.enable()
del d7, cash_stats_by_prev
gc.collect()

# Feature Selection

In [ ]:
# let's review our dataset data types
d1.dtypes.value_counts()

In [ ]:
d1.shape

In [ ]:
# it looks like we have a couple of objects still in our data?
d1.describe(include=[np.object])

In [ ]:
# our outlier variables appear to still be in the format of True or False, so we need to fix this before continuing.
d1['bureau_DAYS_CREDIT_ENDDATE_max_outlier'] = d1['bureau_DAYS_CREDIT_ENDDATE_max_outlier'].map({False:0, True:1})
d1['bureau_DAYS_ENDDATE_FACT_mean_outlier'] = d1['bureau_DAYS_ENDDATE_FACT_mean_outlier'].map({False:0, True:1})

In [ ]:
# because the dataset file is so large, let's use a subsample of the data to evaluate the collinear variables
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

y_temp = d1[['TARGET']]
X_temp = d1.drop(['TARGET'], axis=1)
X_big, X_small, y_big, y_small = train_test_split(X_temp, y_temp, test_size=0.2, random_state=1)

In [ ]:
# let's first make the correlation matrix
corr = X_small.drop(['SK_ID_CURR'], axis=1)
corr_matrix = corr.corr().abs()

In [ ]:
upper_corr = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper_corr.head()

In [ ]:
# let's drop any columns with correlations above 0.9
drop_cols = [column for column in upper_corr.columns if any(upper_corr[column] > 0.9)]
print('Columns to remove: ', len(drop_cols))

In [ ]:
# now we can drop these columns from the full dataset file
dataset = d1.drop(columns = drop_cols)
print('dataset data shape: ', dataset.shape)

In [ ]:
# delete the dataframes we don't need anymore
gc.enable()
del X_temp, X_big, X_small, y_temp, y_big, y_small, corr, corr_matrix, upper_corr, drop_cols
gc.collect()

# Missing Value

In [ ]:
# missing values (in percent)
dataset_missing = (dataset.isnull().sum() / len(dataset)).sort_values(ascending = False)
dataset_missing.head(10)

In [ ]:
# let's remove columns with more than 75% missing data
dataset_missing = dataset_missing.index[dataset_missing > 0.75]
print('Columns with more than 75% missing values: ', len(dataset_missing))

In [ ]:
# let's drop these columns
dataset = dataset.drop(columns = dataset_missing)
print('dataset data shape: ', dataset.shape)

In [ ]:
for dtype, col in (list(zip(dataset.dtypes, dataset.columns))):
    print(dtype, col)

In [41]:
from google.colab import files
dataset.to_csv('dataclean.csv')
files.download('dataclean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>